<a href="https://colab.research.google.com/github/akash-kola-test/python-data-manipulation/blob/main/Data_Manipulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///test.db')

In [36]:
from sqlalchemy.orm import Mapped, mapped_column, DeclarativeBase, Session

In [37]:
class Base(DeclarativeBase):
    pass

In [38]:
class User(Base):
    __tablename__ = 'json_to_sql_table'

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str]
    phone: Mapped[str]
    email: Mapped[str]
    address: Mapped[str]
    region: Mapped[str]
    country: Mapped[str]
    list_col: Mapped[str] = mapped_column(name='list')
    postalzip: Mapped[str]
    currency: Mapped[str]

Base.metadata.create_all(engine)

In [39]:
import json

In [40]:
with open("/content/drive/MyDrive/sample_data_for_assignment.json") as file:
  users_data = json.load(file)


In [41]:
len(users_data["data"])

500

In [42]:
with Session(engine) as session:
  for user in users_data["data"]:
    db_user = User(name=user[0], phone=user[1], email=user[2], address=user[3], region=user[4], country=user[5], list_col=user[6], postalzip=user[7], currency=user[8])
    session.add(db_user)
  session.commit()


In [43]:
import pandas as pd
df = pd.read_sql_table("json_to_sql_table", engine)

In [44]:
df.set_index("id", inplace=True)

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 1 to 500
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       500 non-null    object
 1   phone      500 non-null    object
 2   email      500 non-null    object
 3   address    500 non-null    object
 4   region     500 non-null    object
 5   country    500 non-null    object
 6   list       500 non-null    object
 7   postalzip  500 non-null    object
 8   currency   500 non-null    object
dtypes: object(9)
memory usage: 39.1+ KB


In [66]:
for i in range(0, df.shape[0]):
  series = df.iloc[i]
  series.name = "Data"
  print(pd.DataFrame(series))
  break

                             Data
name              Winifred Branch
phone              1-251-583-4993
email        in.mi@protonmail.com
address    Ap #530-5652 Arcu. Rd.
region             Rio de Janeiro
country                   Nigeria
list                           19
postalzip               66242-403
currency                   $14.59


In [72]:
df["email"].str.split("@", expand=True)[0].apply(lambda x: x + "@gmail.com")

,0
id,
1,in.mi@gmail.com
2,venenatis.lacus@gmail.com
3,orci.in@gmail.com
4,placerat.eget.venenatis@gmail.com
5,lorem@gmail.com
...,...
496,non.quam.pellentesque@gmail.com
497,suscipit.est@gmail.com
498,nibh.lacinia@gmail.com


In [86]:
import re

def clean_postal_zip(postal_zip):
  postal_zip = str(postal_zip)
  postal_zip = re.sub(r'[^0-9]', '', postal_zip)
  return int(postal_zip) if postal_zip else None

postal_zips = df["postalzip"].map(clean_postal_zip)
postal_zips.isna().sum()

0

In [87]:
postal_zips

,postalzip
id,
1,66242403
2,18317
3,8581
4,28826
5,335049
...,...
496,49343
497,393741
498,70801


In [112]:
import re

def phone_ascii(phone):
  phone = str(phone)
  phone = re.sub(r'[^0-9]', '', phone)

  ascii_codes = ""

  for i in range(0, len(phone), 2):
    value = phone[i: i+2]
    if len(value) != 2:
      continue
    int_value = int(value)
    if int_value < 0 or int_value > 127:
      continue
    ascii_codes += chr(int_value)

  return ascii_codes

df["Ascii"] = df["phone"].map(phone_ascii)


In [114]:
df[df["phone"].isin(["(816) 530-4269", "1-811-920-9732"])]

,name,phone,email,address,region,country,list,postalzip,currency,Ascii
id,,,,,,,,,,
8,Rana Schneider,(816) 530-4269,dictum.cursus@outlook.com,387-7160 Aenean Street,Newfoundland and Labrador,Italy,7,59L 5R7,$37.58,QA *E
9,Constance Rocha,1-811-920-9732,ornare.egestas.ligula@aol.ca,724-3665 Ligula. Ave,Bihar,Brazil,15,44258,$23.88, \\tI


In [118]:
df.rename({"Ascii": "coded_phone_number"}, inplace=True, axis=1)

In [119]:
df.columns

Index(['name', 'phone', 'email', 'address', 'region', 'country', 'list',
       'postalzip', 'currency', 'coded_phone_number'],
      dtype='object')